In [31]:
import pandas as pd

training_raw = pd.read_csv(r"C:\Repos\reddit-capstone\reddit-capstone\reddit-capstone\reddit_both_labeled.csv", encoding = "utf-8", low_memory=False)
test_raw = pd.read_csv(r"C:\Repos\reddit-capstone\reddit-capstone\reddit-capstone\upliftingnews_senti_testdata.csv", encoding = "utf-8", low_memory=False)

training_data = training_raw.loc[:,["title", "sentiment"]]
testing_data = test_raw.loc[:,["title"]]


In [40]:
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer
import numpy as np

nltk.download("punkt")

stemmer = PorterStemmer()
def stem_words(words, stemmer):
    stemmed = []
    for w in words:
        stemmed.append(stemmer.stem(w))
    return stemmed

def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    tokens = nltk.word_tokenize(text)
    stems = stem_words(tokens, stemmer)
    return stems

vectorizer = CountVectorizer(analyzer = 'word', tokenizer = tokenize, lowercase = True,
                             stop_words = 'english', max_features = 1000)  # top 1000 features by frequency
# want all possible words in our corpus (labeled/train and unlabeled/test)
corpus_all = vectorizer.fit_transform(list(training_data["title"]) + list(testing_data["title"])).toarray()

corpus_vocab = vectorizer.get_feature_names() # all words

dist = np.sum(corpus_all, axis=0) # freq of tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zande\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test  = train_test_split(corpus_all[:len(training_data)], training_data["sentiment"],
                                                     train_size=0.90, random_state=416)

log_reg_model = LogisticRegression()
log_reg_model = log_reg_model.fit(X=X_train, y=y_train)
y_pred = log_reg_model.predict(X_test)

print(classification_report(y_test, y_pred))


             precision    recall  f1-score   support

          0       0.83      0.83      0.83       487
          1       0.84      0.83      0.83       497

avg / total       0.83      0.83      0.83       984



C:\Users\zande\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [55]:
log_reg_model_2 = LogisticRegression()
log_reg_model_2.fit(X = corpus_all[:len(training_data)], y=training_data["sentiment"])

test_pred = log_reg_model_2.predict(corpus_all[len(training_data):])

for text, sentiment in zip(testing_data["title"], test_pred):
    if sentiment == 0:
        print(sentiment, text)

0 Homeless man's selfless act caught on camera
0 Rhonda Farley says she's just another mom. But those who know her say she is so much more than that. "Saint," "queen," and "great woman" are all superlatives used to describe her.
0 Lamborghini grants wish of dying child to drive in a pink Lamborghini Aventador with Richard Hammond from Top Gear
0 Happening right meow: New York puts bill before both the Senate and Assembly which would ban declawing of cats making it the first state to do so
0 George Zimmerman punched in the face for bragging about killing Trayvon Martin
0 Millions in U.S. Climb Out of Poverty, at Long Last
0 Green Bay police officer buys food for boy accused of shoplifting
0 Office Depot to Close for Thanksgiving
0 The Guardian view on social media: facts need to be labelled as facts | Editorial | Opinion
0 Virtual reality to aid Auschwitz war trials of concentration camp guards
0 Germany launches World's first crowdfunded train service
0 Campaign reminds people of rule 